In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain_community.embeddings import XinferenceEmbeddings
import chromadb
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import List, Dict
import os

import json
from langserve import add_routes


In [2]:
chroma_client = chromadb.HttpClient(host='direct.virtaicloud.com', port=20994)

In [3]:

# 1. 连接数据库
# 获取embeddings模型 - bge
embed = XinferenceEmbeddings(
    server_url="http://direct.virtaicloud.com:28511", model_uid="custom-bge-m3"
)

chroma_client = chromadb.HttpClient(host='direct.virtaicloud.com', port=20994)

# 自定义embedding函数
class MyEmbeddingFunction:
    def __call__(self, input: str) -> list[float]:
        # embed the documents somehow
        print("------执行了--------")
        embeddings = embed.embed_documents([input])
        return embeddings[0]

# 获取集合
collection = chroma_client.get_or_create_collection(name="testDB4", embedding_function=MyEmbeddingFunction())

# 2. 引入大模型
def get_self_model_connect(base_url=None, api_key=None, model_name=None, stream_option=None):
    """
    获取自定义模型连接对象
    """
    # 连接大模型
    llm = ChatOpenAI(
        base_url=base_url,
        openai_api_key=api_key,
        model_name=model_name,
        temperature=0.01,
        max_tokens=512,
        streaming=stream_option
    )
    return llm

llm = get_self_model_connect(
    base_url="http://direct.virtaicloud.com:25933/v1",
    api_key="XX",
    model_name="qwen-vl-chat",
    stream_option=True
)

def customer_retriever(q:str,embed,collection):
    """
    自定义检索器
    @Param q: 查询的问题
    """
    query_embed = embed.embed_query(q)
    results = collection.query(
        query_embeddings=query_embed, # Chroma will embed this for you
        n_results=1 # how many results to return
    )
    # print(results)
    return results


# 多角度转换问题
def get_multi_query(query:str):
    """
    多查询策略
    @param query: 问题
    @return： 多查询结果
    """
    import requests

    # 注意请求路径：常规路径后面添加一个 invoke
    url = " http://direct.virtaicloud.com:42383/intention_rec/invoke"

    data = {"src_question":query}

    # 注意传参格式：外面包一层 input
    response = requests.post(url=url, json={"input": data})
    res = response.json()["output"]["content"]

    # str-->json 转换
    json_decoder = json.JSONDecoder()
    res_multi_query = json_decoder.decode(res)
    
    # print(response.json())
    return res_multi_query

def retrieve_and_answer(question):
    from collections import Counter
    """
    检索并回答
    @Param q: 查询的问题
    """
    print(question)
    # 获取多查询
    res_multi_query = get_multi_query(query=question)
    try:

        src_question = res_multi_query["src_question"]
        new_question1 = res_multi_query["new_question1"]
        new_question2 = res_multi_query["new_question2"]
        new_question3 = res_multi_query["new_question3"]

        results0 = customer_retriever(src_question,embed,collection)
        results1 = customer_retriever(new_question1,embed,collection)
        results2 = customer_retriever(new_question2,embed,collection)
        results3 = customer_retriever(new_question3,embed,collection)
        
        print(results0['documents'][0])
        titles = []
        titles.append(results0["metadatas"][0][0]["title"])
        titles.append(results1["metadatas"][0][0]["title"])
        titles.append(results2["metadatas"][0][0]["title"])
        titles.append(results3["metadatas"][0][0]["title"])
        
        
        # 获取最多的元素
        # frequency_count = Counter(titles)
        # most_frequent = frequency_count.most_common(1)[0][0]
        # TODO: 待完善
        if set(titles) < 2:
            return {
                "content":results['documents'][0],
                "metadatas":results["metadatas"][0][0]
            }
        else: 
            return None
    except Exception as e:
        print("多查询策略失败，请检查接口是否正常")
        results = customer_retriever(question,embed,collection)
        if results["distances"][0][0] != None and results["distances"][0][0] < 0.8:
            return {
                "content":results['documents'][0],
                "metadatas":results["metadatas"][0][0]
            }
        else:
            return None




d:\TOOL\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
def customer_search(query,collection,embed):
    """
    自定义检索器
    @Param q: 查询的问题
    """
    query_embed = embed.embed_query(query)
    results = collection.query(
        query_embeddings=query_embed, # Chroma will embed this for you
        n_results=1 # how many results to return
    )
    # print(results)
    return results


In [5]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document

class MyVectorStoreRetriever(BaseRetriever):
    """
    基于向量数据库的 Retriever 实现
    """

    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        """Retriever 的同步实现"""

        from chromadb import QueryResult
        query_result: QueryResult = customer_search(query,collection,embed)
        print(query_result)
        if query_result:
            docs = query_result["documents"]
            if docs:
                return [Document(page_content=doc[0]) for doc in docs]
        return []


In [6]:
myRetriever = MyVectorStoreRetriever()

In [7]:
myRetriever.invoke(input="感冒")

{'ids': [['c4eef4d6350b465188f899ac19227a7b']], 'distances': [[0.6329997777938843]], 'embeddings': None, 'metadatas': [[{'source': 'https://www.yaozs.com/sms4173/', 'title': '氨咖黄敏胶囊'}]], 'documents': [["'适应症': '适用于缓解普通感冒及流行性感冒引起的发热、头痛、四肢酸痛、打喷嚏、流鼻涕、鼻塞、咽痛等症状。', '规格': '----', '不良反应': '有时有轻度头晕、乏力、恶心、上腹不适、口干、食欲缺乏和皮疹等，可自行恢复。', '用法用量': '口服。成人，一次1～2粒，一日3次。', '禁忌': '严重肝肾功能不全者禁用。', '注意事项': '1. 用药3-7天，症状未缓解，请咨询医师或药师。\\n2. 服用本品期间不得饮酒或含有酒精的饮料。\\n3. 不能同时服用与本品成份相似的其他抗感冒药。\\n4. 前列腺肥大、青光眼等患者以及老年人应在医师指导下使用。\\n5. 肝、肾功能不全者慎用。\\n6. 孕妇及哺乳期妇女慎用。\\n7. 服药期间不得驾驶机、车、船、从事高空作业、机械作业及操作精密仪器。\\n8. 如服用过量或出现严重不良反应，应立即就医。\\n9. 对本品过敏者禁用，过敏体质者慎用。\\n10. 本品性状发生改变时禁止使用。\\n11. 请将本品放在儿童不能接触的地方。\\n12. 儿童必须在成人监护下使用。\\n13. 如正在使用其他药品，使用本品前请咨询医师或药师。',"]], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


[Document(page_content="'适应症': '适用于缓解普通感冒及流行性感冒引起的发热、头痛、四肢酸痛、打喷嚏、流鼻涕、鼻塞、咽痛等症状。', '规格': '----', '不良反应': '有时有轻度头晕、乏力、恶心、上腹不适、口干、食欲缺乏和皮疹等，可自行恢复。', '用法用量': '口服。成人，一次1～2粒，一日3次。', '禁忌': '严重肝肾功能不全者禁用。', '注意事项': '1. 用药3-7天，症状未缓解，请咨询医师或药师。\\n2. 服用本品期间不得饮酒或含有酒精的饮料。\\n3. 不能同时服用与本品成份相似的其他抗感冒药。\\n4. 前列腺肥大、青光眼等患者以及老年人应在医师指导下使用。\\n5. 肝、肾功能不全者慎用。\\n6. 孕妇及哺乳期妇女慎用。\\n7. 服药期间不得驾驶机、车、船、从事高空作业、机械作业及操作精密仪器。\\n8. 如服用过量或出现严重不良反应，应立即就医。\\n9. 对本品过敏者禁用，过敏体质者慎用。\\n10. 本品性状发生改变时禁止使用。\\n11. 请将本品放在儿童不能接触的地方。\\n12. 儿童必须在成人监护下使用。\\n13. 如正在使用其他药品，使用本品前请咨询医师或药师。',")]

In [8]:
from langchain_core.prompts import ChatPromptTemplate
# 可执行的占位符
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_messages([
  ("human", """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
如果上下文中没有出现跟问题相关的信息，请直接回答不知道即可！
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:""")
])

In [9]:
print(prompt.invoke(input={"context": "我的参考上下文", "question":"我是问题"}).messages[0].content)

You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
如果上下文中没有出现跟问题相关的信息，请直接回答不知道即可！
Use three sentences maximum and keep the answer concise.
Question: 我是问题 
Context: 我的参考上下文 
Answer:


In [10]:
# 把检索到的4条上下文的文本使用 \n\n 练成一个大的字符串
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
rag_chain = (
    {"context": myRetriever | format_docs,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [12]:
rag_chain.invoke(input="感冒")

{'ids': [['c4eef4d6350b465188f899ac19227a7b']], 'distances': [[0.6329997777938843]], 'embeddings': None, 'metadatas': [[{'source': 'https://www.yaozs.com/sms4173/', 'title': '氨咖黄敏胶囊'}]], 'documents': [["'适应症': '适用于缓解普通感冒及流行性感冒引起的发热、头痛、四肢酸痛、打喷嚏、流鼻涕、鼻塞、咽痛等症状。', '规格': '----', '不良反应': '有时有轻度头晕、乏力、恶心、上腹不适、口干、食欲缺乏和皮疹等，可自行恢复。', '用法用量': '口服。成人，一次1～2粒，一日3次。', '禁忌': '严重肝肾功能不全者禁用。', '注意事项': '1. 用药3-7天，症状未缓解，请咨询医师或药师。\\n2. 服用本品期间不得饮酒或含有酒精的饮料。\\n3. 不能同时服用与本品成份相似的其他抗感冒药。\\n4. 前列腺肥大、青光眼等患者以及老年人应在医师指导下使用。\\n5. 肝、肾功能不全者慎用。\\n6. 孕妇及哺乳期妇女慎用。\\n7. 服药期间不得驾驶机、车、船、从事高空作业、机械作业及操作精密仪器。\\n8. 如服用过量或出现严重不良反应，应立即就医。\\n9. 对本品过敏者禁用，过敏体质者慎用。\\n10. 本品性状发生改变时禁止使用。\\n11. 请将本品放在儿童不能接触的地方。\\n12. 儿童必须在成人监护下使用。\\n13. 如正在使用其他药品，使用本品前请咨询医师或药师。',"]], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


'感冒是一种常见的疾病，通常由普通感冒或流行性感冒引起。本品适用于缓解感冒症状，如发热、头痛、四肢酸痛、打喷嚏、流鼻涕、鼻塞、咽痛等。使用本品时应注意遵医嘱，不能同时服用与本品成份相似的其他抗感冒药，孕妇及哺乳期妇女慎用。如果出现严重不良反应，应立即就医。'

### 汇总

In [13]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain_community.embeddings import XinferenceEmbeddings
import chromadb
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import List, Dict
import os

from langserve import add_routes

app = FastAPI()

# 1. 连接数据库
# 获取embeddings模型 - bge
embed = XinferenceEmbeddings(
    server_url="http://direct.virtaicloud.com:28511", model_uid="custom-bge-m3"
)

chroma_client = chromadb.HttpClient(host='direct.virtaicloud.com', port=20994)

# 自定义embedding函数
class MyEmbeddingFunction:
    def __call__(self, input: str) -> list[float]:
        # embed the documents somehow
        print("------执行了--------")
        embeddings = embed.embed_documents([input])
        return embeddings[0]

# 获取集合
collection = chroma_client.get_or_create_collection(name="testDB4", embedding_function=MyEmbeddingFunction())

# 2. 引入大模型
def get_self_model_connect(base_url=None, api_key=None, model_name=None, stream_option=None):
    """
    获取自定义模型连接对象
    """
    # 连接大模型
    llm = ChatOpenAI(
        base_url=base_url,
        openai_api_key=api_key,
        model_name=model_name,
        temperature=0.01,
        max_tokens=512,
        streaming=stream_option
    )
    return llm

llm = get_self_model_connect(
    base_url="http://direct.virtaicloud.com:25933/v1",
    api_key="XX",
    model_name="qwen-vl-chat",
    stream_option=False
)

# 3. 生成多种提问方式
def generate_variations(question: str) -> List[str]:
    role_description = "你是一个专业的信息重组专家，你的任务是帮助用户提供多种不同的提问方式，以便从不同的角度获取信息。"
    task_description = "请根据提供的问题，给出至少3种不同的提问方式。每种提问方式都应该尽量覆盖问题的不同方面，同时保持问题的主旨不变。"
    template = f"""{role_description}
    {task_description}
    问题：{question}
    不同的提问方式：
    1. 
    2. 
    3. 
    """
    prompt = PromptTemplate(template=template, input_variables=["question"])
    chain = LLMChain(llm=llm, prompt=prompt)
    variations = chain.run(question=question)
    variations_list = [v.strip() for v in variations.split('\n') if v.strip()]
    return variations_list

def ask_rag_system(question: str) -> List[dict]:
    query_embed = embed.embed_query(question)
    results = collection.query(query_embeddings=query_embed, n_results=1)
    parsed_results = []

    if 'documents' in results and 'metadatas' in results and 'distances' in results:
        for doc, meta, distance in zip(results['documents'][0], results['metadatas'][0], results['distances'][0]):
            parsed_result = {
                'distance': distance,
                'metadata': meta,
                'document': doc
            }
            parsed_results.append(parsed_result)

    return parsed_results

def deduplicate_and_sort_results(results: List[List[Dict[str, any]]]) -> List[Dict[str, any]]:
    flattened_results = [item for sublist in results for item in sublist]
    unique_results_dict = {}
    for result in flattened_results:
        metadata_key = (result['metadata']['source'], result['metadata'].get('title', ''))
        if metadata_key not in unique_results_dict:
            unique_results_dict[metadata_key] = result
        else:
            existing_result = unique_results_dict[metadata_key]
            existing_result['document'] += '\n' + result['document']
    unique_results = list(unique_results_dict.values())
    unique_results.sort(key=lambda x: x['distance'], reverse=True)
    return unique_results

def generate_answer(question: str, answers: str) -> str:
    template = """问题：{question}
    答案：{answers}
    """
    prompt = PromptTemplate(template=template, input_variables=["question", "answers"])
    print(prompt)
    chain = LLMChain(llm=llm, prompt=prompt)
    answer = chain.run(question=question, answers=answers)
    return answer




In [14]:

# 生成提问变化
variations = generate_variations("抗癌药物有哪些？")


d:\TOOL\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
d:\TOOL\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [15]:
variations

['1. 你能列举出所有已知的抗癌药物吗？',
 '2. 你能详细介绍一下抗癌药物的种类和它们的用途吗？',
 '3. 你能列举出一些常用的抗癌药物，并解释它们是如何工作的吗？']

In [16]:
# 向 RAG 系统提问
answers_lists = [ask_rag_system(variation) for variation in variations]

answers_lists

[[{'distance': 1.0236133918091508,
   'metadata': {'source': 'https://www.yaozs.com/sms13012/', 'title': '穿心莲胶囊'},
   'document': "建促进纤溶，降低低切时血粘度效果肯定。无论体外试验还是口服给药，穿心莲均可显著抑制ADP诱导的血小板一相和二相聚集反应。在体外该药对血小板聚集的抑制作用呈高度的量效关系。穿心莲迅速的抗血小板聚集作用，似与增加cAMP含量的抗血小板药物特点相同。它对内源性、外源性凝血途径中的各种凝血因子及凝血酶作用不大，在体内外却均可使优球蛋白溶解时间（ELT）缩短，提示该药可通过某种途径对纤溶起促进作用。\\n6.抗肿瘤：实验证明穿心莲内酯衍生物穿琥氨酸不论大剂量、中剂量，还是小剂量，对肿瘤细胞的生长皆有抑制作用，且随剂量增加作用增强，抑瘤效果确实，稳定。有报道穿心莲能提高机体对肿瘤细胞的免疫反应。通过体外实验发现穿心莲对培养的癌细胞3H-TdR掺入有抑制作用，证实穿心莲对培养的乳腺癌细胞DNA合成具有抑制作用。',"}],
 [{'distance': 0.7225680351257324,
   'metadata': {'source': 'https://www.yaozs.com/sms1301/', 'title': '安康欣胶囊'},
   'document': "'适应症': '用于肺癌、胃癌、肝癌、食道癌、直肠癌、鼻咽癌、乳腺癌、子宫颈癌、恶性淋巴癌、淋巴细胞性白血病、膀胱癌、颅内肿瘤等。', '规格': '0.5g*45粒', '不良反应': '尚不明确。', '用法用量': '口服，饭后温开水送服。一次4-6粒，一日3次。疗程30天。', '禁忌': '孕妇忌用或遵医嘱。', '注意事项': '请注意掌握剂量，勿超剂量使用。', '孕妇及哺乳期妇女用药': '孕妇忌用。', '儿童用药': '儿童必须在成人监护下使用,遵医嘱。', '老人用药': '老人应在专业医师指导下使用。', '药物相互作用': '老人应在专业医师指导下使用。', '药理毒理': '未进行相关实验且无可供参考数据。', '药代动力学': '未进行相关实验且无可供参考数据。', '贮藏': '置于干燥处，密闭，防潮。', '有效期

In [17]:
# 汇总答案
aggregated_answers = deduplicate_and_sort_results(answers_lists)


In [18]:
aggregated_answers

[{'distance': 1.0236133918091508,
  'metadata': {'source': 'https://www.yaozs.com/sms13012/', 'title': '穿心莲胶囊'},
  'document': "建促进纤溶，降低低切时血粘度效果肯定。无论体外试验还是口服给药，穿心莲均可显著抑制ADP诱导的血小板一相和二相聚集反应。在体外该药对血小板聚集的抑制作用呈高度的量效关系。穿心莲迅速的抗血小板聚集作用，似与增加cAMP含量的抗血小板药物特点相同。它对内源性、外源性凝血途径中的各种凝血因子及凝血酶作用不大，在体内外却均可使优球蛋白溶解时间（ELT）缩短，提示该药可通过某种途径对纤溶起促进作用。\\n6.抗肿瘤：实验证明穿心莲内酯衍生物穿琥氨酸不论大剂量、中剂量，还是小剂量，对肿瘤细胞的生长皆有抑制作用，且随剂量增加作用增强，抑瘤效果确实，稳定。有报道穿心莲能提高机体对肿瘤细胞的免疫反应。通过体外实验发现穿心莲对培养的癌细胞3H-TdR掺入有抑制作用，证实穿心莲对培养的乳腺癌细胞DNA合成具有抑制作用。',\n建促进纤溶，降低低切时血粘度效果肯定。无论体外试验还是口服给药，穿心莲均可显著抑制ADP诱导的血小板一相和二相聚集反应。在体外该药对血小板聚集的抑制作用呈高度的量效关系。穿心莲迅速的抗血小板聚集作用，似与增加cAMP含量的抗血小板药物特点相同。它对内源性、外源性凝血途径中的各种凝血因子及凝血酶作用不大，在体内外却均可使优球蛋白溶解时间（ELT）缩短，提示该药可通过某种途径对纤溶起促进作用。\\n6.抗肿瘤：实验证明穿心莲内酯衍生物穿琥氨酸不论大剂量、中剂量，还是小剂量，对肿瘤细胞的生长皆有抑制作用，且随剂量增加作用增强，抑瘤效果确实，稳定。有报道穿心莲能提高机体对肿瘤细胞的免疫反应。通过体外实验发现穿心莲对培养的癌细胞3H-TdR掺入有抑制作用，证实穿心莲对培养的乳腺癌细胞DNA合成具有抑制作用。',"},
 {'distance': 0.7225680351257324,
  'metadata': {'source': 'https://www.yaozs.com/sms1301/', 'title': '安康欣胶囊'},
  'document': "'适应症': '用于肺癌、胃癌、肝癌、食道癌、直肠癌、鼻咽

In [19]:
# 生成最终答案
final_answer = generate_answer("抗癌药物有哪些？", str(aggregated_answers))

input_variables=['answers', 'question'] template='问题：{question}\n    答案：{answers}\n    '


In [20]:
final_answer

'抗癌药物包括穿心莲胶囊、安康欣胶囊等。'

In [21]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
class MyVectorStoreRetriever(BaseRetriever):
    """
    基于向量数据库的 Retriever 实现
    """

    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        """Retriever 的同步实现"""

        variations = generate_variations("抗癌药物有哪些？")
        answers_lists = [ask_rag_system(variation) for variation in variations]
        aggregated_answers = deduplicate_and_sort_results(answers_lists)
        return aggregated_answers


In [22]:
myRetriever = MyVectorStoreRetriever()
question = "抗癌药物有哪些？"
myRetriever.invoke(input=question)

[{'distance': 1.0236133918091508,
  'metadata': {'source': 'https://www.yaozs.com/sms13012/', 'title': '穿心莲胶囊'},
  'document': "建促进纤溶，降低低切时血粘度效果肯定。无论体外试验还是口服给药，穿心莲均可显著抑制ADP诱导的血小板一相和二相聚集反应。在体外该药对血小板聚集的抑制作用呈高度的量效关系。穿心莲迅速的抗血小板聚集作用，似与增加cAMP含量的抗血小板药物特点相同。它对内源性、外源性凝血途径中的各种凝血因子及凝血酶作用不大，在体内外却均可使优球蛋白溶解时间（ELT）缩短，提示该药可通过某种途径对纤溶起促进作用。\\n6.抗肿瘤：实验证明穿心莲内酯衍生物穿琥氨酸不论大剂量、中剂量，还是小剂量，对肿瘤细胞的生长皆有抑制作用，且随剂量增加作用增强，抑瘤效果确实，稳定。有报道穿心莲能提高机体对肿瘤细胞的免疫反应。通过体外实验发现穿心莲对培养的癌细胞3H-TdR掺入有抑制作用，证实穿心莲对培养的乳腺癌细胞DNA合成具有抑制作用。',\n建促进纤溶，降低低切时血粘度效果肯定。无论体外试验还是口服给药，穿心莲均可显著抑制ADP诱导的血小板一相和二相聚集反应。在体外该药对血小板聚集的抑制作用呈高度的量效关系。穿心莲迅速的抗血小板聚集作用，似与增加cAMP含量的抗血小板药物特点相同。它对内源性、外源性凝血途径中的各种凝血因子及凝血酶作用不大，在体内外却均可使优球蛋白溶解时间（ELT）缩短，提示该药可通过某种途径对纤溶起促进作用。\\n6.抗肿瘤：实验证明穿心莲内酯衍生物穿琥氨酸不论大剂量、中剂量，还是小剂量，对肿瘤细胞的生长皆有抑制作用，且随剂量增加作用增强，抑瘤效果确实，稳定。有报道穿心莲能提高机体对肿瘤细胞的免疫反应。通过体外实验发现穿心莲对培养的癌细胞3H-TdR掺入有抑制作用，证实穿心莲对培养的乳腺癌细胞DNA合成具有抑制作用。',"},
 {'distance': 0.7225680351257324,
  'metadata': {'source': 'https://www.yaozs.com/sms1301/', 'title': '安康欣胶囊'},
  'document': "'适应症': '用于肺癌、胃癌、肝癌、食道癌、直肠癌、鼻咽

In [23]:
from langchain_core.prompts import ChatPromptTemplate
# 可执行的占位符
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_messages([
  ("human", """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
如果上下文中没有出现跟问题相关的信息，请直接回答不知道即可！
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:""")
])

In [24]:
rag_chain = (
    {"context": myRetriever,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
rag_chain.invoke(input=question)

'抗癌药物包括穿心莲胶囊、安康欣胶囊等。'

In [26]:
import requests
import json

In [27]:
data = "抗癌药品有哪些"

response = requests.post(url="http://127.0.0.1:8000/rag_asg/invoke", json={"input": data})

print(response.json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /rag_asg/invoke (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002BBE8A0AE50>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

### 自定义解析器

In [28]:
# 参考：https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.base.RunnableGenerator.html
from typing import Any, AsyncIterator, Iterator

from langchain_core.runnables import RunnableGenerator


def gen(input: Iterator[Any]) -> Iterator[str]:
    for token in ["Have", " a", " nice", " day"]:
        yield token


In [64]:
def gen(input: Iterator[Any]) -> Iterator[str]:
    for token in input:
        yield token

In [65]:
runnable = RunnableGenerator(gen)

In [30]:
runnable.invoke(None)  # "Have a nice day"

'Have a nice day'

In [31]:
list(runnable.stream(None))  # ["Have", " a", " nice", " day"]

['Have', ' a', ' nice', ' day']

In [32]:
runnable.batch([None, None])  # ["Have a nice day", "Have a nice day"]

['Have a nice day', 'Have a nice day']

### 自定义解析器2

In [33]:
from langchain_core.messages import AIMessage, AIMessageChunk


# 自定义解析器2
def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()

In [60]:
from typing import Iterable
from langchain_core.messages import AIMessage, AIMessageChunk

def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()


streaming_parse = RunnableGenerator(streaming_parse)

In [66]:
rag_chain = (
    {"context": myRetriever,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | runnable
)

In [67]:
rag_chain.stream(input=question)

<generator object RunnableSequence.stream at 0x000002BBEBC20220>

In [68]:
for chunk in rag_chain.stream(input=question):
    print(chunk, end="|", flush=True)

content='抗' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='癌' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='药物' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='包' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='括穿' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='心' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='莲' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='胶囊、安康欣胶囊等。' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|content='' id='run-f0cd5417-2124-495d-aa53-acacf5fde03e'|